In [1]:
DATASET_data = "rec.mu2e.CRV_wideband_cosmics.CRVWB-000-006-000-012.root"
DATASET_mc_v5 = "nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v5.root"
DATASET_mc_v2 = "nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v2.root"

In [1]:
import time
import os
import re

from mu2etools import *
from mu2etools import wideband
hep.style.use('ATLAS')  # or ATLAS/LHCb2
import hist
from hist import Hist
import mplhep as mplhep
from scipy.stats import binom

In [5]:
processor = wideband.DataProcessor(usexroot=True, treename='CrvWidebandTest/WidebandTree')
ar2 = processor.getEffData("nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v2.root", nfiles=10, data_type=2)
ar5 = processor.getEffData("nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v5.root", nfiles=10, data_type=5)
processor = wideband.DataProcessor(usexroot=True, treename='run')
ar_data = processor.getEffData("rec.mu2e.CRV_wideband_cosmics.CRVWB-000-006-000-012.root", nfiles=10, data_type=-1)

Processing file: root://fndcadoor.fnal.gov:1094/mu2e/tape/phy-nts/nts/mu2e/CRV_wideband_cosmics-mc/config_0011_v2/root/ae/7d/nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v2.001002_00000000.root:CrvWidebandTest/WidebandTree
Processing file: root://fndcadoor.fnal.gov:1094/mu2e/tape/phy-nts/nts/mu2e/CRV_wideband_cosmics-mc/config_0011_v5/root/89/69/nts.mu2e.CRV_wideband_cosmics-mc.config_0011_v5.001005_00000003.root:CrvWidebandTest/WidebandTree
Processing file: root://fndcadoor.fnal.gov:1094/mu2e/tape/phy-rec/rec/mu2e/CRV_wideband_cosmics-noadc/CRVWB-000-006-000/root/e9/92/rec.mu2e.CRV_wideband_cosmics-noadc.CRVWB-000-006-000.001720_000.root:run


In [10]:
ar = ak.concatenate([ar2, ar5, ar_data])

In [ ]:
%%time
#List of variable to export to a skimmed array
varlist=['spillNumber', 'eventNumber','runNumber', 'subrunNumber',
         'trackPEs', 'trackPoints', 'trackChi2', 'trackIntercept', 'trackSlope']

all_layers = np.arange(0,16) 
test_layers = np.arange(2,6) # Test layers are 2 through 6
trig_layers = all_layers[~np.isin(all_layers, test_layers)]
varlist.extend(['nTrigHits', 'PEsTestLayers', 'PEsTrigLayers', "mc"])
#varlist.extend(['PEsTestLayers'])

ar_skim_list = []

#filelist = filelist_mc[0:10] + filelist_data[0:1]
filelist = filelist_mc_v5+filelist_mc_v2+filelist_data[0:5]
for idx, filename in enumerate(filelist):
    if idx%20 == 0:
        print("Processing file: %s"%filename)
    file = uproot.open(filename)
    for ar in uproot.iterate(file, step_size="10MB", 
                                   filter_name=['PEs', varlist, 'coincidencePosX'], 
                                   library='ak', options={"timeout": 7200}):
        # Set PEs to zero for aging and quad-counters
        ak.to_numpy(ar['PEs'])[:,0,0:8] = 0
        ak.to_numpy(ar['PEs'])[:,3,0:8] = 0
        ak.to_numpy(ar['PEs'])[:,7,0:8] = 0
                                
        # Filter out hits below 5PE
        ar_trig_filt = ak.where(ar['PEs'] >= 5, ar['PEs'], 0)
        # Calculate PEs for even and odd layers
        PEs_even_layers = ak.sum(ar_trig_filt[:, :, 0:32], axis=-1).to_numpy()
        PEs_odd_layers = ak.sum(ar_trig_filt[:, :, 32:64], axis=-1).to_numpy()        
        # Interleave the elements from even and odd arrays
        PEs_interleaved = np.concatenate((PEs_even_layers[:, :, np.newaxis], PEs_odd_layers[:, :, np.newaxis]), axis=2)
        # Reshape the interleaved array to match the desired shape
        PEs_stacked = PEs_interleaved.reshape(-1, 16)        
        # Add PEs for even and odd layers to the original array
        ar['PEsAllLayer'] = PEs_stacked
        # Count the number of triggered layers
        ar['nTrigHits'] = ak.sum(ar['PEsAllLayer'][:, trig_layers] > 10, axis=-1)
        # Filter out events with more than 10 triggered layers        
        ar = ar[ar['nTrigHits'] > 8]
        # Extract only PEsTestLayers to save memory
        ar['PEsTestLayers'] = ar['PEsAllLayer'][:, test_layers]
        ar['PEsTrigLayers'] = ar['PEsAllLayer'][:, trig_layers]
        
        pattern = re.compile(r'cosmics-mc\.config.*_v(\d+)')
        match = re.search(pattern, filename)
        
        print(int(match.group(1)))
        if match:
            ar['mc'] = int(match.group(1))
            ar['trackIntercept'] = ar['trackIntercept'] - 20950.0
            # Mimic the trigger paddles
            trigPad_cut = (abs(ar["coincidencePosX"][:,1]+5604) < 20) & (abs(ar["coincidencePosX"][:,4] + 5604) < 20)
            ar = ar[trigPad_cut]
        else:
            ar['mc'] = -1
        
        # Append to the list
        ar_skim_list.append(ar[varlist])

In [ ]:
ar_skim = ak.concatenate(ar_skim_list, axis=0)
# Number of layer hits in the test module
ar_skim['nTestHits'] = ak.sum(ar_skim['PEsTestLayers'] > 10, axis=-1)
# Chi2NDF only if denominator if > 0
ar_skim['trackChi2NDF'] = ak.where(ar_skim['trackPoints'] > 2, ar_skim['trackChi2'] / (ar_skim['trackPoints'] - 2), -999)

In [ ]:
plt.hist(ar_skim[ar_skim['mc']==5]['nTrigHits'], bins=13, range=(0, 13), histtype='step', label='MC: v5', density=True);
plt.hist(ar_skim[ar_skim['mc']==2]['nTrigHits'], bins=13, range=(0, 13), histtype='step', label='MC: v2', density=True);
plt.hist(ar_skim[ar_skim['mc']==-1]['nTrigHits'], bins=13, range=(0, 13), histtype='step', label='Data', density=True);
plt.xlabel('nTrigHits');
plt.legend()

In [ ]:
# Cuts
chi2NDF_lower_limit = 0
chi2NDF_upper_limit = 30

intercept_lower_limit = -700
intercept_upper_limit = -200

#intercept_lower_limit = -520
#intercept_upper_limit = -290

trackPoints_limit = 40
trackPEs_limit = 3000
nTrigHits_limit = 12

# Define the cut conditions
condition_chi2 = (ar_skim["trackChi2NDF"] > chi2NDF_lower_limit) & (ar_skim["trackChi2NDF"] < chi2NDF_upper_limit)
condition_intercept = (ar_skim["trackIntercept"] > intercept_lower_limit) & (ar_skim["trackIntercept"] < intercept_upper_limit)
condition_trackPEs = ar_skim["trackPEs"] < trackPEs_limit
condition_nTrigHits = ar_skim["nTrigHits"] >= nTrigHits_limit
condition_trackPoints = ar_skim["trackPoints"] < trackPoints_limit

# Cut categories
cut_categories = [condition_chi2, 
                  condition_chi2 & condition_intercept, 
                  condition_chi2 & condition_intercept & condition_trackPEs, 
                  condition_chi2 & condition_intercept & condition_trackPEs & condition_nTrigHits, 
                  condition_chi2 & condition_intercept & condition_trackPEs & condition_nTrigHits & condition_trackPoints]

In [ ]:
# Create histograms
axis1 = hist.axis.IntCategory(label='NHits', name='nhits', categories=[1, 2, 3, 4])
axis2 = hist.axis.StrCategory(label='Cuts', name='cut', categories=["trackChi2", "intercept", "trackPEs", "nTrigHits", "all"])
axis3 = hist.axis.IntCategory(label='Data type', name='dtype', categories=[-1, 1, 2, 3, 4, 5])
axis4 = hist.axis.IntCategory(label='NLayers', name='layer', categories=[1, 2, 3, 4])

h_nTrigHits = Hist(hist.axis.Regular(6, 8, 14, name="nTrigHits"), axis1, axis2, axis3)
h_PEsTestLayers = Hist(hist.new.Reg(100, 0, 200, name="PEsTestLayers"), axis1, axis2, axis3)
h_trackChi2NDF = Hist(hist.new.Reg(100, 0, 100, name="trackChi2NDF"), axis1, axis2, axis3)
h_trackPEs = Hist(hist.new.Reg(100, 1000, 5000, name="trackPEs"), axis1, axis2, axis3)
h_trackPoints = Hist(hist.new.Reg(80, 0, 80, name="trackPoints"), axis1, axis2, axis3)
h_trackIntercept = Hist(hist.new.Reg(80, -1000, 100, name="trackIntercept"), axis1, axis2, axis3)
h_trackSlope = Hist(hist.new.Reg(80, -1, 1, name="trackSlope"), axis1, axis2, axis3)

h_trackPEsHit = Hist(hist.new.Reg(100, 0, 500, name="trackPEsHit"), axis1, axis2, axis3)

h_PEsLayer = Hist(hist.new.Reg(100, 0, 500, name="PEsLayer", flow=True), axis1, axis2, axis3, axis4)
h_PEsLayerSort = Hist(hist.new.Reg(100, 0, 500, name="PEsLayerSort", flow=True), axis1, axis2, axis3, axis4)

histogram_list = [h_nTrigHits, h_PEsTestLayers, h_trackChi2NDF, 
                  h_trackPEs, h_trackPoints, h_trackIntercept, h_trackSlope]

# Iterate over nhits values and fill the histograms
for data_type in axis3:
    dtype_condition = (ar_skim["mc"] == data_type)
    for nhits in axis1:
        hits_condition = ar_skim['nTestHits'] <= nhits
        for idx, cut in enumerate(axis2):

            cut_condition = cut_categories[idx] & hits_condition & dtype_condition

            for histogram in histogram_list:
                var_name = histogram.axes[0].label
                if(var_name == "PEsTestLayers"):
                    arr_fill = ak.flatten(ar_skim[cut_condition][var_name])
                else:
                    arr_fill = ar_skim[cut_condition][var_name]
                histogram.fill(arr_fill, nhits=nhits, cut=cut, dtype=data_type)

            h_trackPEsHit.fill(ar_skim[cut_condition]['trackPEs']/ar_skim[cut_condition]['trackPoints'], nhits=nhits, cut=cut, dtype=data_type)

            for layer in axis4:
                h_PEsLayer.fill(ar_skim[cut_condition]['PEsTestLayers'][:,layer-1], nhits=nhits, cut=cut, dtype=data_type, layer=layer)
                h_PEsLayerSort.fill(ak.sort(ar_skim[cut_condition]['PEsTestLayers'], axis=1)[:,layer-1], nhits=nhits, cut=cut, dtype=data_type, layer=layer)

histogram_list.extend([h_trackPEsHit, h_PEsLayerSort, h_PEsLayer])

In [ ]:
for layer in range(0,4):
    plt.figure()
    h_PEsLayerSort[:, 3, "all", 2, layer].plot(flow="sum", label='Layer %d: MC-v2'%layer, density=True)
    h_PEsLayerSort[:, 3, "all", 5, layer].plot(flow="sum", label='Layer %d: MC-v5'%layer, density=True)
    h_PEsLayerSort[:, 3, "all", 0, layer].plot(flow="sum", label='Layer %d: Data'%layer, density=True)
#    plt.yscale('log');
    plt.legend()
    plt.xlabel("PEsLayerSort")
    plt.xlim(0,300)

for layer in range(0,4):
    plt.figure()
    h_PEsLayer[:, 3, "all", 2, layer].plot(flow="sum", label='Layer %d: MC-v2'%layer, density=True)
    h_PEsLayer[:, 3, "all", 5, layer].plot(flow="sum", label='Layer %d: MC-v5'%layer, density=True)
    h_PEsLayer[:, 3, "all", 0, layer].plot(flow="sum", label='Layer %d: Data'%layer, density=True)
#    plt.yscale('log');
    plt.xlabel("PEsLayer")
    plt.legend()    
#    plt.xlim(0,300)


In [ ]:
def get_extrapolated_efficiency(cut, data_type, nfold):
    h = h_PEsLayer[:, 3, cut, data_type, :]
    # Normalize histogram to unity
    normalized_hist = np.mean(h, axis=1) / np.mean(h, axis=1).sum()
    # Calculate cumulative distribution function (CDF)
    cdf = np.cumsum(normalized_hist)
    cdf = binom.cdf(3-nfold, 4, 1 - cdf)
    return cdf

In [ ]:
def plot_efficiency(nfold, ismc):

    h = h_PEsLayerSort[:, 3, "intercept", ismc, nfold]
    # Normalize histogram to unity
    normalized_hist = h.values() / h.sum(flow=True)

    # Calculate cumulative distribution function (CDF)
    cdf = np.cumsum(normalized_hist)
    cdf = np.clip(cdf, 1e-10, 1 - 1e-10)

    # Calculate the error bars for the CDF
    error_cdf = np.sqrt(cdf * (1 - cdf) / h.sum(flow=True))

    # Plot the normalized histogram and CDF with error bars on the same axis
    fig, ax1 = plt.subplots(figsize=(8, 5))

    # Plot the second lowest PEs
    mplhep.histplot(h, ax=ax1, label='PEs layer: sorted', flow="sum")
    # Create a second y-axis
    ax2 = ax1.twinx()
    ax2.fill_between(h.axes[0].centers, cdf - error_cdf, cdf + error_cdf, color='red', alpha=0.2, label='Meas. %d/4 inefficiency'%(4-nfold))
    ax2.plot(h.axes[0].centers, cdf, 'r-', linewidth=0.5)

    cdf_est = get_extrapolated_efficiency("all", ismc, nfold)
    ax2.plot(h.axes[0].centers, cdf_est, 'b-', alpha=0.7, label='Extrap. %d/4 inefficiency'%(4-nfold))

    # Set labels for both y-axes
    ax1.set_ylabel('Counts', color='blue')
    ax2.set_ylabel('Cumulative Probability', color='red')
    
    ax2.axhline(y=1E-4, color='red', linestyle='--', label='CRV requirement', linewidth=1)


    plt.legend(loc='upper left')
    ax2.set_yscale('log')
    ax1.set_yscale('log')
    plt.xlim(0, 200);
    plt.ylim(1E-6, 10);

In [ ]:
for nfold in range(0,4):
    plt.figure()
    plot_efficiency(nfold, True);
    plt.title("MC")
    
    plt.figure()
    plot_efficiency(nfold, False);
    plt.title("Data")

In [ ]:
base_cuts = cut_categories[4] & (abs(ar_skim["trackSlope"]) < 0.1)
# Extract arrays from data and MC samples
data_layer = ar_skim[base_cuts & ~ar_skim["mc"]]['PEsTestLayers']
mc_layer = ar_skim[base_cuts & ar_skim["mc"]]['PEsTestLayers']

plt.hist(ak.sort(data_layer, axis=1)[:,1], bins=20, range=(50, 200), histtype='step', density=True, label='Data');
plt.hist(ak.sort(mc_layer, axis=1)[:,1], bins=20, range=(50, 200), histtype='step', density=True, label='MC');
plt.legend();
plt.xlabel('PEs in the 2nd lowest layer')

for layer in range(0,4):
    plt.figure()
    plt.hist(data_layer[:,layer], bins=20, range=(50, 200), histtype='step', density=True, label='Data');
    plt.hist(mc_layer[:,layer], bins=20, range=(50, 200), histtype='step', density=True, label='MC');
    plt.legend();
    plt.xlabel('PEs in layer %d'%layer);

In [ ]:
#Just a cross-check

# Plot MC
plot_efficiency(1, False);

#Plot Data
h, edges = np.histogram(ak.sort(ar_skim[cut_categories[1] & (~ar_skim["mc"])]['PEsTestLayers'], axis=1)[:,1].to_numpy(), bins=1000, range=(0, 2000))
cumulative = np.cumsum(h).astype(float)  # Convert to float
cumulative /= cumulative[-1]
plt.plot(edges[:-1], cumulative, "--g", label='Data')
plt.legend()
plt.yscale('log');
plt.xlim(0,140);

In [ ]:
h = h_PEsLayerSort[:, 3, "all", True, 1]
cdf_est = get_extrapolated_efficiency("all", True, 1)
plt.plot(h.axes[0].centers, cdf_est, 'b-', alpha=0.7, label='Extr. 3/4 inefficiency: MC')
plt.yscale('log');
plt.xlim(0,140);

h = h_PEsLayerSort[:, 3, "all", False, 1]
cdf_est = get_extrapolated_efficiency("all", False, 1)
plt.plot(h.axes[0].centers, cdf_est, 'g-', alpha=0.7, label='Extrap. 3/4 inefficiency: Data')
plt.yscale('log');
plt.xlim(0,140);
plt.legend()

#Plot MC
h, edges = np.histogram(ak.sort(ar_skim[cut_categories[4] & (ar_skim["mc"])]['PEsTestLayers'], axis=1)[:,1].to_numpy(), bins=1000, range=(0, 2000))
cumulative = np.cumsum(h).astype(float)  # Convert to float
cumulative /= cumulative[-1]
plt.plot(edges[:-1], cumulative, '--', color='orange',  label='Meas. 3/4: MC')
plt.legend()
plt.yscale('log');
plt.xlim(0,140);

#Plot Data
h, edges = np.histogram(ak.sort(ar_skim[cut_categories[4] & (~ar_skim["mc"])]['PEsTestLayers'], axis=1)[:,1].to_numpy(), bins=100, range=(0, 200))
cumulative = np.cumsum(h).astype(float)  # Convert to float
cumulative /= cumulative[-1]
plt.plot(edges[:-1], cumulative, "r--", label='Meas. 3/4: Data')
plt.legend()
plt.yscale('log');
plt.xlim(0,140);

plt.xlabel('Threshold [PEs]')


ex_eff_tyler=[8.42576148e-06, 1.45093773e-05, 4.81819494e-05, 1.40458939e-04,
 5.39816619e-04, 2.53147644e-03, 1.80419720e-02, 1.07752884e-01,
 3.61844511e-01, 6.86125430e-01, 8.78211946e-01, 9.52824329e-01,
 9.78828587e-01,]
meas_eff_tyler=[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
 1.84094256e-04, 1.28865979e-03, 1.71207658e-02, 1.08615611e-01,
 3.68004418e-01, 6.81332842e-01, 8.62665685e-01, 9.32805596e-01,
 9.54712813e-01]
x_tyler=range(10,140, 10)
#plt.plot(x_tyler, ex_eff_tyler, '-', label='Extr. 3/4: MC (Tyler)')
#plt.plot(x_tyler, meas_eff_tyler, '-', label='Meas. 3/4: MC (Tyler)')
plt.legend();

In [ ]:
#plt.hist(ar_skim[ar_skim['mc']==True]['trackIntercept'], bins=1000, range=(20000, 21000), histtype='step');
plt.figure(figsize=(18, 6))  # Adjust the width and height as needed

base_cuts = (abs(ar_skim["trackSlope"]) < 1.5)

#plt.hist(ar_skim[~ar_skim['mc'] & base_cuts]['trackIntercept'], bins=1000, range=(-1000, 0), histtype='step', density=True, label="Data");
#plt.hist(-1*ar_skim[ar_skim['mc'] & base_cuts]['trackIntercept']-1000-5, bins=1000, range=(-1000, 0), histtype='step', density=True, label="MC");
plt.hist(ar_skim[ar_skim['mc'] & base_cuts]['trackIntercept']+24, bins=1000, range=(-1000, 0), histtype='step', density=True, label="MC");
plt.legend()
plt.xlabel("trackIntercept")

plt.figure()

plt.hist(ar_skim[(ar_skim['mc']==False)]['trackChi2NDF'], bins=1000, range=(0, 100), histtype='step', density=True, label="Data");
plt.hist(ar_skim[(ar_skim['mc']==True)]['trackChi2NDF'], bins=1000, range=(0, 100), histtype='step', density=True, label="MC");
plt.legend()
plt.yscale('log')
plt.xlabel("trackChi2");

In [ ]:
for idx, histogram in enumerate(histogram_list[:-2]):
    ncols = 3
    ax_id = idx%ncols
    if ax_id == 0:
        fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(20, 5))

    h = histogram[:, 3, "trackChi2", True]
    mplhep.histplot(h, ax=axes[ax_id], label="MC, Events: %d"%h.to_numpy(flow=True)[0].sum(), density=True, flow="sum")

    h = histogram[:, 3, "trackChi2", False]
    mplhep.histplot(h, ax=axes[ax_id], label="Data, Events: %d"%h.to_numpy(flow=True)[0].sum(), density=True, flow="sum")

#    h = histogram[:, 1, "trackChi2", False]
#    hplot = mplhep.histplot(h, ax=axes[ax_id], label="Data <2/4, Events: %d"%h.to_numpy(flow=True)[0].sum(), density=True, flow="sum", yerr=False)
    
    axes[ax_id].set_xlabel(histogram.axes[0].label)
    axes[ax_id].legend()
#    axes[ax_id].set_yscale('log')

Distributions before any cuts 

In [ ]:
#Apply cuts
ar_skim['nTestHits'] = ak.sum(ar_skim['PEsTestLayers'] > 10, axis=-1)
ar_3l = ar_skim[cut_categories[4] & (ar_skim['nTestHits']<3) & (~ar_skim['mc'])]  # less than 3 hits
ar_4e = ar_skim[cut_categories[4] & (ar_skim['nTestHits']==4) & (~ar_skim['mc'])] # equal to 4 hits

In [ ]:
ar_skim_fields = ar_skim.fields.copy()
ar_skim_fields.remove('PEsTestLayers')
#ar_skim_fields.remove('PEsTrigLayers')

In [ ]:
pd.set_option('display.max_rows', None)
ak.to_dataframe(ar_3l[ar_skim_fields])

In [ ]:
eff = len(ar_3l)/len(ar_4e)
print("Efficiency at 10 PE: %.2e"%eff)